In [1]:
import os
import pandas as pd
from Bio import SeqIO
import sys

In [2]:
sys.path.append('../two_stages_classifier/training/')
import utils
from utils import precision_recall

In [3]:
os.chdir("/fs/ess/PAS0439/MING/cilates_fungi_classifier/outputs/downstream_analysis/benchmark/")

In [4]:
seqorigin = pd.read_csv("/fs/ess/PAS0439/MING/cilates_fungi_classifier/testset_seq_origin.csv")

In [5]:
testing_pro = list(pd.read_csv("../../../dataset_proka_benchmark.csv").id)
testing_fungi = list(pd.read_csv("../../../dataset_fungi_benchmark.csv").files)
testing_protozoa = list(pd.read_csv("../../../dataset_protozoa_benchmark.csv").id)
testing_sags = list(seqorigin.query('category == "SAGs"').genome)
testing_genomes = testing_pro + testing_fungi + testing_protozoa + testing_sags
testing_set = seqorigin[seqorigin.genome.isin(testing_genomes)]

In [6]:
tiara = pd.read_csv('test_5kb_tiara.txt', sep = "\t")
tiara_pro = list(tiara[tiara.class_fst_stage.isin(["prokarya", "archaea", "bacteria"])].sequence_id)
tiara_euk = list(tiara.query("class_fst_stage == 'eukarya'").sequence_id)
tiara_unknown = list(set(tiara.sequence_id) - set(tiara_euk) - set(tiara_pro)) # tiara annotation as unknown or organelle 


/tmp/slurmtmp.25314419/ipykernel_183268/1188852451.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  tiara = pd.read_csv('test_5kb_tiara.txt', sep = "\t")


In [7]:
eukrep_pro = []
eukrep_euk = []
records = SeqIO.parse("test_eukrep_euk.fa", "fasta") 
for record in records:
    eukrep_euk.append(str(record.id))
    
records = SeqIO.parse("test_eukrep_pro.fa", "fasta") 
for record in records:
    eukrep_pro.append(str(record.id))

In [8]:
tiara_out = {}
eukrep_out = {}
Y = {}
for index, row in testing_set.iterrows():
    cate = row["category"]
    seq = row['seq']
    if cate == "prokaryotes":
        Y[seq] = 0
    else:
        Y[seq] = 1
        
for f in tiara_euk:
    tiara_out[f] = 1

for f in tiara_pro:
    tiara_out[f] = 0
    
for f in tiara_unknown:
    try:
        if Y[f] == 0:
            tiara_out[f] = 1
        elif Y[f] == 1:
            tiara_out[f] = 0
    except KeyError:
        continue
        
        
for f in eukrep_pro:
    eukrep_out[f] = 0

for f in eukrep_euk:
    eukrep_out[f] = 1
    
    

In [9]:
tiara_out_df = pd.DataFrame.from_dict(tiara_out, orient = "index").rename(columns = {0:"tiara_out"})
eukrep_out_df = pd.DataFrame.from_dict(eukrep_out, orient = "index").rename(columns = {0:"eukrep_out"})
Y_df = pd.DataFrame.from_dict(Y, orient = "index").rename(columns = {0:"Y"})

In [10]:
tmp = pd.merge(tiara_out_df, eukrep_out_df, left_index = True, right_index = True)
tmp1 = pd.merge(tmp, Y_df, left_index = True, right_index = True)
testing_set_res = pd.merge(testing_set, tmp1, left_on = "seq", right_index = True)

In [11]:
testing_set_res

,genome,seq,category,tiara_out,eukrep_out,Y
0,Diplodinium.dentatum.SAG3,Diplodinium.dentatum.SAG3_5_BT_18068_fragment_1,SAGs,1,1,1
1,Diplodinium.dentatum.SAG3,Diplodinium.dentatum.SAG3_5_BT_18068_fragment_2,SAGs,0,0,1
2,Diplodinium.dentatum.SAG3,Diplodinium.dentatum.SAG3_5_BT_18068_fragment_3,SAGs,0,0,1
3,Diplodinium.dentatum.SAG3,Diplodinium.dentatum.SAG3_5_BT_18068_fragment_4,SAGs,1,0,1
4,Diplodinium.dentatum.SAG3,Diplodinium.dentatum.SAG3_5_BT_18068_fragment_5,SAGs,1,0,1
...,...,...,...,...,...,...
693042,GCA_016125265.1,WGMC01000008.1_fragment_7,prokaryotes,0,0,0
693043,GCA_016125265.1,WGMC01000008.1_fragment_8,prokaryotes,0,0,0
693044,GCA_016125265.1,WGMC01000033.1_fragment_1,prokaryotes,0,0,0
693045,GCA_016125265.1,WGMC01000012.1_fragment_1,prokaryotes,0,0,0


In [12]:
testing_set_res.to_csv("/fs/ess/PAS0439/MING/cilates_fungi_classifier/testset_res_tiara_eukrep.csv", index = None)

In [13]:
# tiara 
precision_recall(list(testing_set_res.tiara_out), list(testing_set_res.Y))

Class 0: Precision: 86.86%| Recall: 99.18%
Class 1: Precision: 99.05%| Recall: 85.08%


In [14]:
print(f'tiara Class 0 F1: {86.86*99.18*2/(86.86 + 99.18)}')
print(f'tiara Class 1 F1: {99.05*85.08*2/(85.08 + 99.05)}')

tiara Class 0 F1: 92.61207052246829
tiara Class 1 F1: 91.5350458914897


In [15]:
# eukrep
precision_recall(list(testing_set_res.eukrep_out), list(testing_set_res.Y))

Class 0: Precision: 88.07%| Recall: 98.25%
Class 1: Precision: 98.03%| Recall: 86.77%


In [16]:
print(f'eukrep Class 0 F1: {88.07*98.25*2/(88.07 + 98.25)}')
print(f'eukrep Class 1 F1: {98.03*86.77*2/(86.77 + 98.03)}')

eukrep Class 0 F1: 92.8818967367969
eukrep Class 1 F1: 92.05695995670995


In [17]:
testing_res_guteuk = pd.read_csv("/fs/ess/PAS0439/MING/cilates_fungi_classifier/test_res_stage1.csv")

In [18]:
guteuk_out = testing_res_guteuk[testing_res_guteuk.seq.isin(list(testing_set_res.seq))]

In [19]:
precision_recall(list(guteuk_out.ensembled_out), list(guteuk_out.Y))

Class 0: Precision: 99.34%| Recall: 99.61%
Class 1: Precision: 99.61%| Recall: 99.34%


In [20]:
print(f'guteuk Class 0 F1: {99.34*99.61*2/(99.61 + 99.34)}')
print(f'guteuk Class 1 F1: {99.61*99.34*2/(99.34 + 99.61)}')

guteuk Class 0 F1: 99.47481678813773
guteuk Class 1 F1: 99.47481678813773


In [21]:
guteuk_out.to_csv("/fs/ess/PAS0439/MING/cilates_fungi_classifier/testset_res_guteuk.csv", index = None)